# Notebook showcasing use of Google Cloud GenAI(Palm API) + Doc AI to extract information from documents

In [ ]:
#install dependencies
!pip install google-cloud-aiplatform --upgrade
#!apt-get install poppler-utils
!pip install google-cloud-core
!pip install google-cloud-documentai
!pip install google-cloud-storage
!pip install simplejson
#!pip install pdf2image

In [ ]:
#import libraries
from google.api_core.client_options import ClientOptions
from google.cloud import documentai

from PIL import Image, ImageDraw
import os
import pandas as pd
from IPython.display import display, Image
from pdf2image import convert_from_path, convert_from_bytes

### Authentication 

In [ ]:
#Run only if using Google Colab Notebooks
from google.colab import auth as google_auth
google_auth.authenticate_user()

## Review the PDF File

In [ ]:
file_path='./sample_data/US_PTO_Sample.pdf'
from IPython.display import IFrame
IFrame(file_path, width=800, height=700)

### Use GCP Document AI to OCR the PDF

In [ ]:
# Define function to OCR the PDF using Document AI
def process_document_sample(
    project_id: str,
    location: str,
    processor_id: str,
    file_path: str,
    mime_type: str,
    field_mask: str = None,
):

    client = documentai.DocumentProcessorServiceClient()

    name = client.processor_path(project_id, location, processor_id)

    # Import the file into memory
    with open(file_path, "rb") as image:
        image_content = image.read()

    # Load the image content
    raw_document = documentai.RawDocument(content=image_content, mime_type=mime_type)


    request = documentai.ProcessRequest(
        name=name, raw_document=raw_document
    )

    result = client.process_document(request=request)


    document = result.document

    # Read the text recognition output from the processor
    return(document.text)



### Use this for PDF Files

In [ ]:
#For PDF Docs
ocr_output = process_document_sample(
  project_id="398507275014",
  location="us",
  processor_id="2fb6b1be15c7f2d",
    mime_type = 'application/pdf',
    field_mask = None,
  file_path= file_path
)

### Use this for TIFF files

In [ ]:
#For TIFF docs
ocr_output = process_document_sample(
  project_id="398507275014",
  location="us",
  processor_id="2fb6b1be15c7f2d",
    mime_type = 'image/tiff',
    field_mask = None,
  file_path="./genai_demo_data/demo_data.tiff"
)

In [ ]:
#Print the first 1000 characters of the OCR output
print(ocr_output[:1000])

In [20]:
### Run the OCR results above through the Vertex AI GenAI/PALM Model to extact entities

In [21]:

import vertexai
from vertexai.preview.language_models import TextGenerationModel

def predict_large_language_model_sample(
    project_id: str,
    model_name: str,
    temperature: float,
    max_decode_steps: int,
    top_p: float,
    top_k: int,
    content: str,
    location: str = "us-central1",
    tuned_model_name: str = "",
    ) :
    """Predict using a Large Language Model."""
    vertexai.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained(model_name)
    if tuned_model_name:
      model = model.get_tuned_model(tuned_model_name)
    response = model.predict(
        content,
        temperature=temperature,
        max_output_tokens=max_decode_steps,
        top_k=top_k,
        top_p=top_p,)
    print(f"Response from Model: {response.text}")


### Enter the prompt to be used for entity etxraction from the document

In [19]:
prompt_suffix = '''What is the name of the inventor(s)?'''

In [ ]:
ocr_text = ocr_output+prompt_suffix
ocr_text

In [ ]:
llm_output = predict_large_language_model_sample("jsb-alto", "text-bison@001", 0.2, 256, 0.8, 40, 
                                    ocr_text, 
                                    "us-central1")

In [ ]:
print(llm_output['content'])